## 📓 Step2 — Cooperative Neural Networks (Colab Notebook)

This notebook is organized into two simple sections:

1. **Environment Setup (run once at the start)**  
   - Installs required packages.  
   - Clones the original ONE_NeurIPS2018 repo and this refactored repo.  
   - Moves into the correct working directory (`Step2`).  
   👉 Must be run before any experiments.

2. **Part1 & Part2 Experiments (run commands only)**  
   - Each subgroup is executed by a single run command cell.  
   - Training and evaluation logic is contained inside the refactored scripts (`train_threecomb.py`, `train_threes_into_three.py`, `train_one_into_three.py`, `train_b_one_into_three.py`, etc).  
   - No manual imports or extra code are needed in the notebook itself.

---

⚠️ **Important Notes**
- This repo does **not** contain pretrained weights.  
  Each viewer must **train and generate their own checkpoints** before running cooperative experiments.  
- Logger automatically writes headers and results — no manual header cells are required.  


##📚 References
- **Reference Article:** **[Xu et al. *Knowledge Distillation On the Fly Native Ensemble* (ONE) NeurIPS 2018.](https://arxiv.org/pdf/1806.04606)**  
- **Original GitHub Repository:** [Lan1991Xu/ONE_NeurIPS2018](https://github.com/Lan1991Xu/ONE_NeurIPS2018)

##Run once at the start of the notebook:

In [ ]:
# Environment setup
!pip install ipython-autotime
%load_ext autotime

# 1. Clone original repo
!git clone https://github.com/Lan1991Xu/ONE_NeurIPS2018.git

# 2. Clone this repo (contains only changed files)
!git clone https://github.com/SemyonKim/Neural-Network-Exercises.git

# 3. Move into repo root (Step2 folder)
%cd Neural-Network-Exercises/ONE_KnowledgeDistillation/TwoStageCooperativeNN/Step2

#Part1

## Simple 1‑Branch Model (Baseline)

This experiment evaluates a simplified version of the ONE method by editing the original
ResNet and CIFAR training scripts to use a **single branch** only.

- **Goal:** Establish a baseline accuracy for ResNet32+ONE with 1 branch on CIFAR100.
- **Changes:**
  1. Edited `resnet.py` to remove extra branches, leaving only `layer3_1` and `classifier3_1`.
  2. Edited `cifar_one.py` to adapt training loop for single branch outputs.
- **Training setup:** 160 epochs, CIFAR100 dataset, ResNet32 depth.
- **Result:** Final test accuracy ≈ **70.12%**.

In [ ]:
!python models/train_onebranch.py \
    -a one_resnet \
    --dataset cifar100 \
    --depth 32 \
    --epochs 160 \
    --consistency_rampup 80 \
    --gpu-id 0 \
    --schedule 151 225 \
    --gamma 0.1 \
    --wd 1e-4 \
    --checkpoint checkpoints/cifar100/ONE-32-rampup


==> Preparing dataset cifar100
Files already downloaded and verified
==> creating model 'one_resnet'
    Total params: 0.47M

Epoch: [1 | 160] LR: 0.100000
Processing |################################| (391/391)   || Loss: 8.3428 |LossKL: 0.0000 | top1_C1:  5.3720 | top1_t:  5.3720 
Processing |################################| (100/100)| Loss:  0.0000 | top1_C1:  10.2700 | top1_t:  10.2700

Epoch: [2 | 160] LR: 0.100000
Processing |################################| (391/391)   || Loss: 7.0092 |LossKL: 0.0000 | top1_C1:  15.2880 | top1_t:  15.2880 
Processing |################################| (100/100)| Loss:  0.0000 | top1_C1:  18.9700 | top1_t:  18.9700

Epoch: [3 | 160] LR: 0.100000
Processing |################################| (391/391)   || Loss: 5.8876 |LossKL: 0.0000 | top1_C1:  25.3200 | top1_t:  25.3200 
Processing |################################| (100/100)| Loss:  0.0000 | top1_C1:  26.1500 | top1_t:  26.1500

Epoch: [4 | 160] LR: 0.100000
Processing |#####################

##Simple 3‑Branch Model
This experiment extends the baseline ONE method by modifying the original
ResNet and CIFAR training scripts to use three parallel branches plus an ensemble output.
- **Goal**: Evaluate cooperative learning with ResNet32+ONE using 3 branches on CIFAR100.
- **Changes**:
    1. Edited resnet.py to define layer3_1, layer3_2, layer3_3 and their classifiers.

    2. Edited cifar_one.py (refactored as train_threebranch.py) to adapt training and testing loops for three branch outputs plus ensemble.

- **Training setup**: 10 epochs, CIFAR100 dataset, ResNet32 depth, consistency ramp‑up enabled, resuming from the 1‑branch baseline checkpoint.

- **Result**:
    - Branch accuracies: 73.36%, 72.43%, 72.05%
    - Ensemble accuracy: 74.90%
    - Demonstrates how ensemble predictions provide a cooperative gain over individual branches, slightly boosting accuracy compared to the single‑branch baseline.

In [ ]:
!python models/train_threebranch.py \
    -a one_resnet \
    --dataset cifar100 \
    --depth 32 \
    --epochs 10 \
    --consistency_rampup 80 \
    --gpu-id 0 \
    --schedule 151 225 \
    --gamma 0.1 \
    --wd 1e-4 \
    --checkpoint checkpoints/cifar10/ONE-32-rampup \
    --resume /content/model_best.pth.tar \
    --evaluate

==> Preparing dataset cifar100
Files already downloaded and verified
==> creating model 'one_resnet'
    Total params: 1.19M
==> Resuming from checkpoint..

Evaluation only
Processing |################################| (100/100)| Loss:  0.0000 | top1_C1:  73.3600 |top1_C2:  72.4300|top1_C3: 72.0500 |top1_t:  74.9000
 Test Loss:  0.00000000, Test Acc1:  73.36, Test Acc2:  72.43, Test Acc3:  72.05, Test Accen:  74.90
time: 9.96 s (started: 2022-05-15 08:31:28 +00:00)


##Simple 5‑Branch Model
This experiment further extends the ONE method by modifying the original
ResNet and CIFAR training scripts to use **five parallel branches** plus an ensemble output.

- **Goal**: Evaluate cooperative learning with ResNet32+ONE using 5 branches on CIFAR10.
- **Changes**:
    1. Edited resnet.py to define layer3_1 … layer3_5 and their classifiers, with ensemble weighting.
    2. Edited cifar_one.py to adapt training and testing loops for five branch outputs plus ensemble.

- **Training setup**: 300 epochs, CIFAR10 dataset, ResNet32 depth, consistency ramp‑up enabled, resuming from 1‑branch pretrained weights copied into all 5 branches.

- **Result**: Shows how ensemble accuracy improves over individual branches, demonstrating stronger cooperative gains compared to 1‑branch and 3‑branch models.

In [ ]:
!python models/train_fivebranch.py \
    -a one_resnet \
    --dataset cifar10 \
    --depth 32 \
    --epochs 300 \
    --consistency_rampup 80 \
    --gpu-id 0 \
    --schedule 151 225 \
    --gamma 0.1 \
    --wd 1e-4 \
    --checkpoint checkpoints/cifar100/ONE-32-rampup \
    --resume /content/model_best.pth.tar \
    --evaluate

##5‑Branch Model with Voting
This experiment extends the simple 5‑branch **ONE‑ResNet** by adding a voting mechanism at the output stage.

- **Architecture**: Five parallel branches are trained simultaneously, each producing its own logits.

- **Special feature**: In addition to the standard ensemble (weighted sum of all branches), the model computes **multiple voting combinations** — subsets of branches (pairs, triples, quadruples, and singles) — to evaluate cooperative performance under different voting scenarios.

- **Purpose**: This design allows analysis of how different branch subsets contribute to accuracy, and whether smaller ensembles can rival or complement the full 5‑branch ensemble.

- **Outcome**: Provides deeper insight into cooperative learning dynamics, showing how ensemble diversity and voting strategies affect final performance compared to individual branches or the full ensemble.

In [ ]:
!python models/train_fivebranch_vote.py \
    -a one_resnet \
    --dataset cifar10 \
    --depth 32 \
    --epochs 300 \
    --consistency_rampup 80 \
    --gpu-id 0 \
    --schedule 151 225 \
    --gamma 0.1 \
    --wd 1e-4 \
    --checkpoint checkpoints/cifar100/ONE-32-rampup \
    --resume /content/model_best.pth.tar \
    --evaluate

## 3‑Branch Model → 5‑Branch Model
This experiment explores transferring pretrained weights from a 3‑branch **ONE‑ResNet** into a **5‑branch version**.

- **Architecture**: ResNet‑32 with five parallel branches plus ensemble.

- **Special feature**: Initialization copies weights from a previously trained 3‑branch model, expanding them into five branches.

- **Two resume options**:

    - --resume → loads the checkpoint from the **3‑branch model**. This provides the pretrained weights that are transferred into the new 5‑branch architecture.

    - --resume0 → loads the checkpoint from the **5‑branch model**. This ensures the optimizer state and training configuration are consistent with the expanded architecture.

- **Purpose**: To test whether knowledge learned in a smaller cooperative setup (3 branches) can be effectively reused and scaled to a larger cooperative model (5 branches).

- **Outcome**: Demonstrates how pretrained cooperative weights accelerate convergence and improve ensemble accuracy compared to training a 5‑branch model from scratch.

In [ ]:
!python train_three_into_five.py \
    -a one_resnet \
    --dataset cifar10 \
    --depth 32 \
    --epochs 100 \
    --consistency_rampup 80 \
    --gpu-id 0 \
    --schedule 151 225 \
    --gamma 0.1 \
    --wd 1e-4 \
    --checkpoint checkpoints/cifar10/Part1 \
    --resume /content/checkpoint.pth.tar \
    --resume0 /content/model_best5.pth.tar \
    --evaluate

#Part2

##3s3 → 3

###Combine Baseline+One‑Branch Models
- This experiment combines **three separate one‑branch baseline models** into a single cooperative 3‑branch model.

- **What we did:**

    - Edited `resnet.py` to define 3 branches (layer3_1 … layer3_3) with control weights.

    - Refactored `cifar_one.py` into `train_threecomb.py` to load three pretrained one‑branch checkpoints (`resume1, resume2, resume3`) alongside the main 3‑branch cooperative checkpoint (`resume`).

- **Why we did it**:  
To test whether multiple independently trained one‑branch baselines can be merged into a stronger cooperative ensemble.

- **Outcome**:  
Evaluation compares original cooperative outputs vs. new combined outputs. Results show how merging pretrained baselines into a cooperative framework affects branch and ensemble accuracies.

In [ ]:
!python train_threecomb.py \
    -a one_resnet \
    --dataset cifar10 \
    --depth 32 \
    --epochs 75 \
    --consistency_rampup 80 \
    --gpu-id 0 \
    --schedule 55 225 \
    --gamma 0.1 \
    --wd 1e-4 \
    --checkpoint checkpoints/cifar100/Part2 \
    --resume /content/model_best3.pth.tar \
    --resume1 /content/model_besta.pth.tar \
    --resume2 /content/model_bestc.pth.tar \
    --resume3 /content/model_bestd.pth.tar \
    --evaluate

==> Preparing dataset cifar10
Files already downloaded and verified
==> creating model 'one_resnet'
    Total params: 1.18M
    Total params: 1.18M
    Total params: 1.18M
    Total params: 1.18M
==> Resuming from checkpoint..

Evaluation only
Processing |################################| (100/100)| Loss:  0.0000 | top1_C1:  64.4900 |top1_C2:  65.6200|top1_C3: 65.7100 |top1_t:  66.5600 | LossNew:  0.0000 | top1_C1New:  94.1500 |top1_C2New:  93.6300|top1_C3New: 93.6400 |top1_tNew:  94.6100
 Test Loss:  0.00000000, Test Acc1:  64.49, Test Acc2:  65.62, Test Acc3:  65.71, Test AccEn:  66.56, Test LossNew:  0.00000000, Test Acc1New:  94.15, Test Acc2New:  93.63, Test Acc3New:  93.64, Test AccEnNew:  94.61


###Merge Three One‑Branch Models
- **What we did:**
    - Reused `resnet_threecomb.py` (3‑branch cooperative ResNet).
    - Refactored `cifar_one.py` into `train_threes_into_three.py`.
    - Loaded three separate one‑branch checkpoints (`resume1, resume2, resume3`) and merged them into a single cooperative 3‑branch model (`resume`).

- **Why we did it:**  
To test whether multiple independently trained one‑branch models can be merged into a stronger cooperative ensemble.

- **Outcome:**  
Evaluation compares branch accuracies and ensemble accuracy after merging. This shows how combining pretrained one‑branch models into a cooperative framework affects performance.

In [ ]:
!python train_threes_into_three.py \
    -a one_resnet \
    --dataset cifar100 \
    --depth 32 \
    --epochs 75 \
    --consistency_rampup 80 \
    --gpu-id 0 \
    --schedule 55 225 \
    --gamma 0.1 \
    --wd 1e-4 \
    --checkpoint checkpoints/cifar100/Part2 \
    --resume /content/model_best.pth.tar \
    --resume1 /content/model_besta.pth.tar --resume1_id 1 \
    --resume2 /content/model_bestb.pth.tar --resume2_id 2 \
    --resume3 /content/model_bestc.pth.tar --resume3_id 3 \
    --evaluate

==> Preparing dataset cifar100
Files already downloaded and verified
==> creating model 'one_resnet'
    Total params: 1.19M
==> Resuming from checkpoint..
Model loaded successfully!

Epoch: [3 | 77] LR: 0.010000
Processing |################################| (391/391)   || Loss: 5.7367 |LossKL: 0.0513 | top1_C1:  89.4640 | top1_C2:  46.0860|top1_C3:  41.6200| top1_t:  79.5680 
Processing |################################| (100/100)| Loss:  0.0000 | top1_C1:  70.9000 |top1_C2:  59.8600|top1_C3: 57.3100 |top1_t:  70.9900

Epoch: [4 | 77] LR: 0.010000
Processing |################################| (391/391)   || Loss: 2.8964 |LossKL: 0.0320 | top1_C1:  89.9820 | top1_C2:  70.1880|top1_C3:  68.3440| top1_t:  86.4780 
Processing |################################| (100/100)| Loss:  0.0000 | top1_C1:  71.0300 |top1_C2:  64.7500|top1_C3: 63.2000 |top1_t:  71.8800

Epoch: [5 | 77] LR: 0.010000
Processing |################################| (391/391)   || Loss: 2.3227 |LossKL: 0.0296 | top1_C1:  9

##1 → 3

###Expand Single‑Branch Model (A)
- **What we did**:
    - Reused `resnet_threecomb.py` (3‑branch cooperative ResNet).
    - Refactored `cifar_one.py` into `train_one_into_three.py`.
    - Loaded a single‑branch checkpoint (resume) and expanded it into a 3‑branch cooperative model (resume0).

- **Why we did it**:  
To test whether a pretrained single‑branch model can be effectively expanded into a cooperative 3‑branch ensemble.

- **Outcome**:  
Evaluation compares branch accuracies and ensemble accuracy after expansion. This shows how cooperative learning benefits from initializing with a strong single‑branch baseline.

In [ ]:
!python train_one_into_three.py \
    -a one_resnet \
    --dataset cifar100 \
    --depth 32 \
    --epochs 120 \
    --consistency_rampup 80 \
    --gpu-id 0 \
    --schedule 151 225 \
    --gamma 0.1 \
    --wd 1e-4 \
    --checkpoint checkpoints/cifar100/Part2 \
    --resume /content/model_best1.pth.tar \
    --resume0 /content/model_best3.pth.tar \
    --evaluate

###Single Branch Inside Cooperative Model (B)
- **What we did:**

    - Reused `resnet_threecomb.py` (3‑branch cooperative ResNet).
    - Refactored `cifar_one.py` into `train_b_one_into_three.py`.
    - Loaded a single‑branch checkpoint (resume) and expanded it into a 3‑branch cooperative model (resume0).

- **Why we did it: **
To test whether a single‑branch model can be directly expanded into a cooperative 3‑branch ensemble without additional pretrained branches.

- **Outcome:  **
Evaluation compares branch accuracies and ensemble accuracy after expansion. This shows how cooperative learning behaves when starting from a single‑branch baseline only.

In [ ]:
!python train_b_one_into_three.py \
    -a one_resnet \
    --dataset cifar10 \
    --depth 32 \
    --epochs 200 \
    --consistency_rampup 80 \
    --gpu-id 0 \
    --schedule 394 435 \
    --gamma 0.1 \
    --wd 1e-4 \
    --checkpoint checkpoints/cifar100/Part2 \
    --resume /content/model_best1.pth.tar \
    --resume0 /content/model_best3.pth.tar \
    --evaluate

==> Preparing dataset cifar10
170499072it [00:05, 29348046.69it/s]                   
Extracting ./data/cifar-10-python.tar.gz to ./data
==> creating model 'one_resnet'
    Total params: 1.18M
==> Resuming from checkpoint..
Model loaded successfully!

Epoch: [295 | 494] LR: 0.010000
Processing |################################| (391/391)   || Loss: 7.5192 |LossKL: 0.0004 | top1_C1:  52.8640 | top1_C2:  52.8680|top1_C3:  52.8820| top1_t:  52.8680 
Processing |################################| (100/100)| Loss:  0.0000 | top1_C1:  61.8900 |top1_C2:  61.8700|top1_C3: 61.8800 |top1_t:  61.8700

Epoch: [296 | 494] LR: 0.010000
Processing |################################| (391/391)   || Loss: 4.1842 |LossKL: 0.0001 | top1_C1:  63.9140 | top1_C2:  63.9340|top1_C3:  63.9340| top1_t:  63.9380 
Processing |################################| (100/100)| Loss:  0.0000 | top1_C1:  66.5300 |top1_C2:  66.5800|top1_C3: 66.6400 |top1_t:  66.5700

Epoch: [297 | 494] LR: 0.010000
Processing |##############